In [1]:
##########################################################
# Manejo de Datos
# Proyecto 
# Equipo: 
##########################################################
import pandas as pd
import sqlite3
import re
class Inegi:
	""" Clase Inegi, contiene un archivo tipo csv con datos recabados por el INEGI """

	##########################################################
	# Metod constructor
	##########################################################
	def __init__(self, name_file):
		self.__name_file=name_file


	##########################################################
	# Pregunta 1
	#
	##########################################################
	def poblacion_total_entidad(self):
		""" Calcula la población total para cada entidad.
		:returns: dict, 
		"""
		
		Ine_D=pd.read_csv(self.__name_file , index_col=0, encoding='latin-1')
		P_total=Ine_D.groupby('NOM_ENT ')['POBTOT'].sum()
		Respuesta_1=P_total.to_dict()
		return Respuesta_1

	##########################################################
	# Pregunta 2
	#
	##########################################################
	def ditritos_poblacion_indigena(self):
		""" Obtiene los distritos que contienen población indígena de cada entidad
		:returns: dict, 
		"""
		Ine_D=pd.read_csv(self.__name_file , index_col=0, encoding='latin-1')
		I_total=Ine_D.groupby('NOM_ENT ')['INDIGENA'].value_counts().unstack(fill_value=0)
		I_perentidad=I_total['SI']
		Respuesta_2=I_perentidad.to_dict()
		return Respuesta_2


	##########################################################
	# Pregunta 3
	#
	##########################################################
	def poblacion_total_cero_dos(self):
		""" Calcula la población total de 0 a 2 años de cada entidad
		:returns: dict,
		"""
		Ine_D=pd.read_csv(self.__name_file , index_col=0, encoding='latin-1')
		I_total=Ine_D.groupby('NOM_ENT ')['P_0A2'].sum()
		Respuesta_3=I_total.to_dict()
		return Respuesta_3


	##########################################################
	# Pregunta 4
	#
	##########################################################
	def poblacion_3_mas_hli(self):
		""" Calcula la población total de 3 años a más que hablan alguna lengua indígena de cada entidad
		:return: dict,
		"""
		Ine_D=pd.read_csv(self.__name_file , index_col=0, encoding='latin-1')
		I_total=Ine_D.groupby('NOM_ENT ')['P3YM_HLI'].sum()
		Respuesta_4=I_total.to_dict()
		return Respuesta_4

	##########################################################
	# Pregunta 5
	#
	##########################################################
	def prom_3_14_nae(self):
		""" Calcula la población total de 3 a 14 años que no asisten a la escuela
		:return: dict,
		"""
		Ine_D=pd.read_csv(self.__name_file , index_col=0, encoding='latin-1')
		I_total=Ine_D.groupby('NOM_ENT ')[['P3A5_NOA','P6A11_NOA','P12A14NOA']].sum()
		I_total['P3A14_NOA']=I_total.sum(axis=1)
		I_total=I_total['P3A14_NOA']
		I_total2=Ine_D.groupby('NOM_ENT ')[['P_3A5','P_6A11','P_12A14']].sum()
		I_total2['P3A14']=I_total2.sum(axis=1)
		I_total2=I_total2['P3A14']
		Promedio=I_total/I_total2
		Respuesta_5=Promedio.to_dict()
		return Respuesta_5


	##########################################################
	# Agregar y consultar datos
	#
	##########################################################
	def agrega_datos(self):
		""" Lee los datos del archivo csv y los agrega a la tabla 'entidad' de la base de datos
		:return: bool, True si se agregaron los datos, False en otro caso
		"""
		try:
			Ine_D=pd.read_csv(self.__name_file , index_col=0, encoding='latin-1')
			conn = sqlite3.connect('resumen_inegi.db')
			cursor = conn.cursor()
			cursor.execute(' DROP TABLE IF EXISTS entidad;')
			cursor.execute('''
			CREATE TABLE entidad(
				id INTEGER PRIMARY KEY,
				nombre TEXT,
				poblacion_total INTEGER,
				poblacion_total_fem INTEGER,
				poblacion_total_masc INTEGER
				)
			''')
			P_total=Ine_D.groupby(['ENTIDAD','NOM_ENT '])[['POBTOT','POBFEM', 'POBMAS']].sum()
			P_total=P_total.reset_index()
			df_as_lists = P_total.apply(lambda x: x.tolist(), axis=1)
			pattern = "[\[\]]"
		
		except:
			return False
		else:
			try:
				for i in range(0,len(df_as_lists)):
					original_string=str(df_as_lists.iloc[i])
					new_string = re.sub(pattern, "", original_string)
					sql='''INSERT  INTO entidad
					(id, nombre, poblacion_total, poblacion_total_fem, poblacion_total_masc)
					VALUES ({});'''.format(new_string);
					cursor.execute(sql)

				conn.commit()
				conn.close()
				return True
			except:
				return False

		

	def consulta_datos(self, id):
		""" Consulta los datos correspondientes a la entidad correspondiente al id
		:return: dict, 
		"""
		a=self.agrega_datos()
		if a==False:
			print("ups")
			exit()
		conn = sqlite3.connect('resumen_inegi.db')
		cursor = conn.cursor()
		cursor.execute('SELECT * FROM entidad;')
		employee_dict = {}
		for row in cursor:
			ID=row[0]
			nombre = row[1]
			poblacion_total = row[2]
			poblacion_total_fem = row[3]
			poblacion_total_masc = row[4]
			employee_dict[ID] ={ nombre:
			[poblacion_total, poblacion_total_fem, poblacion_total_masc]}
		conn.close()
		return employee_dict.get(id)
		

		
		


if __name__ == '__main__':
	# Aca puedes probar tus funciones
	mi_inegi = Inegi("INE_DISTRITO_2020.csv")
	#print(mi_inegi.poblacion_total_entidad())
	#print(mi_inegi.ditritos_poblacion_indigena())
	#print(mi_inegi.poblacion_total_cero_dos())
	#print(mi_inegi.poblacion_3_mas_hli())
	print(mi_inegi.prom_3_14_nae())
	#print(mi_inegi.consulta_datos(1))
	#print(mi_inegi.agrega_datos())

{'Aguascalientes': 0.13802655977225362, 'Baja California': 0.16813162974740029, 'Baja California Sur': 0.12332236637904277, 'Campeche': 0.1244420356176851, 'Chiapas': 0.17353367395045058, 'Chihuahua': 0.16970783930069516, 'Ciudad de MÃ©xico': 0.10561177579455291, 'Coahuila de Zaragoza': 0.14270565978623673, 'Colima': 0.1281443966868456, 'Durango': 0.14494954689065823, 'Guanajuato': 0.14908637245378145, 'Guerrero': 0.10829238585953872, 'Hidalgo': 0.11449564475848206, 'Jalisco': 0.15163177590525653, 'MichoacÃ¡n de Ocampo': 0.15063008588431798, 'Morelos': 0.13716681097728534, 'MÃ©xico': 0.1388543744522407, 'Nayarit': 0.12826494772029529, 'Nuevo LeÃ³n': 0.14720048488509968, 'Oaxaca': 0.1115458718617036, 'Puebla': 0.12411084575692648, 'QuerÃ©taro': 0.13560477575257662, 'Quintana Roo': 0.16276638240033028, 'San Luis PotosÃ\xad': 0.09928528605398414, 'Sinaloa': 0.12629938057715132, 'Sonora': 0.1448712624238391, 'Tabasco': 0.09332254798297758, 'Tamaulipas': 0.15301034498638671, 'Tlaxcala': 0.1